In [1]:
import pandas as pd
import numpy as np

In [126]:
prod = pd.read_csv("01.Pruduct.csv")
sear = pd.read_csv("02.Search1.csv")
sess = pd.read_csv("05.Session.csv")
mast = pd.read_csv("06.Master.csv")
print(prod.shape)
print(sear.shape)
print(sess.shape)
print(mast.shape)

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(5024906, 8)
(2884943, 4)
(2712907, 9)
(847652, 5)


In [127]:
sess = sess.dropna()
print(sess.isnull().sum())
print(sess.shape)
sess['TOT_SESS_HR_V'] = sess['TOT_SESS_HR_V'].map(lambda x: x.replace(",", ""))
sess.head()

CLNT_ID            0
SESS_ID            0
SESS_SEQ           0
SESS_DT            0
TOT_PAG_VIEW_CT    0
TOT_SESS_HR_V      0
DVC_CTG_NM         0
ZON_NM             0
CITY_NM            0
dtype: int64
(2698696, 9)


,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,1890,mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,1604,mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,1035,mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298,mobile,Seoul,Seoul


In [128]:
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

In [129]:
act_buy = prod[['SESS_ID']]
act_buy['ACTION'] = 'buy'

act_sear = sear[['SESS_ID']]
act_sear['ACTION'] = 'search'

act = pd.concat([act_buy, act_sear], axis=0, sort=False)
print(act['ACTION'].unique())
act = act.drop_duplicates()
act = act.groupby('SESS_ID').agg(lambda x: x.tolist())

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


['buy' 'search']


In [130]:
#act['ACTION'].unique

act['ACTION'] = act['ACTION'].map(lambda x: "".join(x) if len(x)<2 else ",".join(x))
print(act['ACTION'].unique())   
print(act.tail())

['buy' 'buy,search']
              ACTION
SESS_ID             
10977908         buy
10977924         buy
10977932  buy,search
10977936  buy,search
10977941         buy


In [131]:
df = pd.merge(sess, act, left_on='SESS_ID', right_on='SESS_ID', how='left')
df.shape

(2698696, 10)

In [132]:
df = df[['CLNT_ID', 'SESS_ID','SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']]
df.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,mobile,"buy,search"
1,5873599,6616320,21,105.0,1604,mobile,"buy,search"
2,5873599,5886172,40,41.0,632,mobile,"buy,search"
3,5873884,1050889,15,160.0,1035,mobile,"buy,search"
4,5874461,10298270,5,13.0,298,mobile,buy


In [9]:
from sklearn.preprocessing import LabelEncoder

encode_DVC = LabelEncoder()
encode_DVC.fit(df['DVC_CTG_NM'])

df['DVC_CTG_NM'] = encode_DVC.transform(df['DVC_CTG_NM'])

'''
encode_ACT = LabelEncoder()
encode_ACT.fit(df['ACTION'])

df['ACTION'] = encode_ACT.transform(df['ACTION'])

df.head()
'''

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,1,1
1,5873599,6616320,21,105.0,1604,1,1
2,5873599,5886172,40,41.0,632,1,1
3,5873884,1050889,15,160.0,1035,1,1
4,5874461,10298270,5,13.0,298,1,0


In [133]:
act = df.iloc[:,2:].groupby('ACTION').agg(['size','sum'])
act[:5]

SESS_SEQ            TOT_PAG_VIEW_CT              TOT_SESS_HR_V  \
               size        sum            size          sum          size   
ACTION                                                                      
buy         1332423  212394228         1332423   68510581.0       1332423   
buy,search  1366273  270168235         1366273  162504052.0       1366273   

                                                              DVC_CTG_NM  \
                                                          sum       size   
ACTION                                                                     
buy         2984442632881598887561492839118133521904855602...    1332423   
buy,search  1890160463210353409763099317414378610383121104...    1366273   

                                                               
                                                          sum  
ACTION                                                         
buy         mobilemobiledesktopmobilemobilemobilemobilemob...  
buy,search  mobilemobilemobilemobilemobilemobilemobilemobi...

In [ ]:
act = df.iloc[:,2:].groupby('ACTION').agg(['size','mean'])
act[:5]

In [ ]:
encode_ACT = LabelEncoder()
encode_ACT.fit(df['ACTION'])

df['ACTION'] = encode_ACT.transform(df['ACTION'])

df.head()

In [10]:
from sklearn.cluster import MiniBatchKMeans, KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

### 먼저 action컬럼만 clustering

In [11]:
cols = ['ACTION']

In [19]:
clusterer = KMeans(n_clusters=2)
preds = clusterer.fit_predict(df[cols])
print(clusterer.score(df[cols]))

-0.0


In [20]:
df['CLUSTER'] = preds
df.shape

(2698696, 8)

In [21]:
df

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,1,1,0
1,5873599,6616320,21,105.0,1604,1,1,0
2,5873599,5886172,40,41.0,632,1,1,0
3,5873884,1050889,15,160.0,1035,1,1,0
4,5874461,10298270,5,13.0,298,1,0,1
...,...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,1,0,1
2698692,3501155,4636571,86,44.0,2090,0,1,0
2698693,3501397,582632,128,164.0,2435,1,0,1
2698694,3573271,2328407,9,5.0,986,0,0,1


In [22]:
data = df.groupby(['CLUSTER','ACTION']).agg({'ACTION':'size'})
data.head()

,,ACTION
CLUSTER,ACTION,
0,1,1366273
1,0,1332423


In [23]:
fulldata = df.iloc[:,2:].groupby('CLUSTER').agg(['size','sum'])

In [24]:
fulldata

SESS_SEQ            TOT_PAG_VIEW_CT              TOT_SESS_HR_V  \
            size        sum            size          sum          size   
CLUSTER                                                                  
0        1366273  270168235         1366273  162504052.0       1366273   
1        1332423  212394228         1332423   68510581.0       1332423   

                                                           DVC_CTG_NM  \
                                                       sum       size   
CLUSTER                                                                 
0        1890160463210353409763099317414378610383121104...    1366273   
1        2984442632881598887561492839118133521904855602...    1332423   

                   ACTION           
             sum     size      sum  
CLUSTER                             
0        1164063  1366273  1366273  
1        1137282  1332423        0

In [25]:
###
# CSV파일로 저장하기
fulldata.to_csv("actionCluster_index.csv")
df.to_csv("cluster.csv")

In [34]:
use = df.groupby(['CLUSTER','DVC_CTG_NM']).agg({'DVC_CTG_NM':'size'})
use = use.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
use
## 별 차이가 없네.. 사용한거 다 비슷비슷혀.. 

DVC_CTG_NM
CLUSTER DVC_CTG_NM            
0       0            15.249441
        1            84.301234
        2             0.449325
1       0            15.085299
        1            84.474975
        2             0.439725

In [42]:
df

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,1,1,0
1,5873599,6616320,21,105.0,1604,1,1,0
2,5873599,5886172,40,41.0,632,1,1,0
3,5873884,1050889,15,160.0,1035,1,1,0
4,5874461,10298270,5,13.0,298,1,0,1
...,...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,1,0,1
2698692,3501155,4636571,86,44.0,2090,0,1,0
2698693,3501397,582632,128,164.0,2435,1,0,1
2698694,3573271,2328407,9,5.0,986,0,0,1


## action과 사용기기로 clustering

In [ ]:
dff = df

In [123]:
cols=['ACTION','DVC_CTG_NM']
clusterer = KMeans(n_clusters=5)
preds = clusterer.fit_predict(dff[cols])
print(clusterer.score(dff[cols]))

-0.0


In [124]:
df['CLUSTER'] = preds
df.shape

(5, 11)

In [125]:
df

SESS_SEQ            TOT_PAG_VIEW_CT              TOT_SESS_HR_V  \
            size        sum            size          sum          size   
CLUSTER                                                                  
0        1131423  205594451         1131423   62963256.0       1131423   
1        1151785  263134178         1151785  149756457.0       1151785   
2         208349    6136731          208349   11940107.0        208349   
3         201000    6799777          201000    5547325.0        201000   
4           6139     897326            6139     807488.0          6139   

                                                           DVC_CTG_NM  \
                                                       sum       size   
CLUSTER                                                                 
0        2984442881598887561492839118133521904855602156...    1131423   
1        1890160463210353409763099317414378610383121104...    1151785   
2        8172938341156216607721472327114029610402792482...     208349   
3        2637323111321384528885652741070233207920367005...     201000   
4        1081122581397362360258052680328955517266485424...       6139   

                   ACTION          CLUSTER  
             sum     size      sum          
CLUSTER                                     
0        1137282  1131423        0       0  
1        1151785  1151785  1151785       2  
2              0   208349   208349       4  
3              0   201000        0       3  
4          12278     6139     6139       1

In [117]:
df = df.iloc[:,2:].groupby('CLUSTER').agg(['size','sum'])
df

SESS_SEQ            TOT_PAG_VIEW_CT              TOT_SESS_HR_V  \
            size        sum            size          sum          size   
CLUSTER                                                                  
0        1131423  205594451         1131423   62963256.0       1131423   
1        1151785  263134178         1151785  149756457.0       1151785   
2         208349    6136731          208349   11940107.0        208349   
3         201000    6799777          201000    5547325.0        201000   
4           6139     897326            6139     807488.0          6139   

                                                           DVC_CTG_NM  \
                                                       sum       size   
CLUSTER                                                                 
0        2984442881598887561492839118133521904855602156...    1131423   
1        1890160463210353409763099317414378610383121104...    1151785   
2        8172938341156216607721472327114029610402792482...     208349   
3        2637323111321384528885652741070233207920367005...     201000   
4        1081122581397362360258052680328955517266485424...       6139   

                   ACTION           
             sum     size      sum  
CLUSTER                             
0        1137282  1131423        0  
1        1151785  1151785  1151785  
2              0   208349   208349  
3              0   201000        0  
4          12278     6139     6139

In [122]:
use = df.

ValueError: Grouper for 'DVC_CTG_NM' not 1-dimensional

In [121]:
group = df.groupby(['CLUSTER','DVC_CTG_NM'])['ACTION']
group
#functionlist = ['size', 'mean', 'std']
#dvc_clus = group['TOT_PAG_VIEW_CT','ACTION'].agg(functionlist)
#dvc_clus

ValueError: Grouper for 'DVC_CTG_NM' not 1-dimensional

In [65]:
sess = pd.read_csv('Clustered_Session.csv')
sess.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,mobile,"buy,search",4
1,5873599,6616320,21,105.0,1604,mobile,"buy,search",4
2,5873599,5886172,40,41.0,632,mobile,"buy,search",0
3,5873884,1050889,15,160.0,1035,mobile,"buy,search",3
4,5874461,10298270,5,13.0,298,mobile,buy,0


In [66]:
cluster = df['CLUSTER'] 
print(cluster.head())


0    1
1    1
2    1
3    1
4    0
Name: CLUSTER, dtype: int32


In [67]:
sess['CLUSTER'] = cluster
sess

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,mobile,"buy,search",1
1,5873599,6616320,21,105.0,1604,mobile,"buy,search",1
2,5873599,5886172,40,41.0,632,mobile,"buy,search",1
3,5873884,1050889,15,160.0,1035,mobile,"buy,search",1
4,5874461,10298270,5,13.0,298,mobile,buy,0
...,...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,mobile,buy,0
2698692,3501155,4636571,86,44.0,2090,desktop,"buy,search",2
2698693,3501397,582632,128,164.0,2435,mobile,buy,0
2698694,3573271,2328407,9,5.0,986,desktop,buy,3


In [71]:
master = pd.read_csv('06.Master.csv')
master.head()

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
1,62282,여자 플라워덧신 2족선물세트 17403-2set,속옷/양말/홈웨어,여성양말류,여성일반양말
2,61729,88A2933253배트맨스웨트티 - 블랙 / 130,유아동의류,유아의류상의,영유아티셔츠/탑
3,61537,"닥터마틴 아드리안 블랙, 체리레드 - 02_체리레드 / 250mm(6)",패션잡화,남성화,남성부츠
4,58820,여성 그레이 스트라이프 퍼프 소매 블라우스 (128865YQ33) - 회색(앤틱실버...,남성의류,남성의류상의,남성남방셔츠


In [74]:
product = prod[['CLNT_ID','SESS_ID','PD_C']]
product['CLUSTER'] = cluster
product
product[product['CLUSTER'].isnull()].head()
product = product.dropna()
print(product.tail())

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


         CLNT_ID   SESS_ID    PD_C  CLUSTER
2698691  5726893  10703446  405458      0.0
2698692  5736441   1530644  837751      2.0
2698693  5737065   4105677  707145      0.0
2698694  5741277   4033875  699794      3.0
2698695  5768908   8517578  519123      3.0


In [75]:
data3 = product.groupby('CLNT_ID').agg(lambda x: x.tolist())
data3

,SESS_ID,PD_C,CLUSTER
CLNT_ID,,,
25,"[5317297, 6059256, 8113243, 9506206]","[598634, 715072, 715121, 715121]","[1.0, 0.0, 0.0, 0.0]"
32,"[759569, 759569, 1136737, 1136737]","[31982, 31974, 48881, 48881]","[3.0, 1.0, 0.0, 1.0]"
33,"[10548225, 8694850, 10300429]","[605825, 613253, 806941]","[1.0, 1.0, 0.0]"
56,"[2108568, 2108568]","[425529, 425529]","[0.0, 0.0]"
58,[4328876],[146435],[0.0]
...,...,...,...
6592536,[1347403],[240908],[1.0]
6592544,[8270794],[706193],[1.0]
6592551,"[1352463, 1352463, 1107655]","[815842, 829686, 751413]","[0.0, 0.0, 1.0]"


In [99]:
newdata = pd.merge(master, product, left_on='PD_C', right_on='PD_C', how='left')
newdata.shape
newdata

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_ID,SESS_ID,CLUSTER
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,1489284.0,2719478.0,1.0
1,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,4671890.0,2659813.0,3.0
2,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,559585.0,2330751.0,0.0
3,62282,여자 플라워덧신 2족선물세트 17403-2set,속옷/양말/홈웨어,여성양말류,여성일반양말,2820547.0,733100.0,0.0
4,61729,88A2933253배트맨스웨트티 - 블랙 / 130,유아동의류,유아의류상의,영유아티셔츠/탑,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3097372,400721,[피핀] 클레르 박스 롱티셔츠 103764 - ivory / FREE ◈85564075◈,여성의류,여성의류상의,여성티셔츠/탑,NaN,NaN,NaN
3097373,400382,[바니플랫]여성플랫/BNF63011EN - 브라운 / 245 ◈83660832◈,패션잡화,여성화,여성플랫,NaN,NaN,NaN
3097374,400197,섀르반 올인원 수영복 (K14D5DM231) - BL/110,시즌스포츠,수영/물놀이,아동수영복,1428278.0,5969955.0,1.0
3097375,399225,링클플리츠주름밴딩스커트(CFHM1SK8821) - 블랙 / FREE,여성의류,여성의류하의,여성스커트,NaN,NaN,NaN


In [100]:
print(newdata.isnull().sum())

PD_C             0
PD_NM            0
CLAC1_NM         0
CLAC2_NM         0
CLAC3_NM         0
CLNT_ID     398681
SESS_ID     398681
CLUSTER     398681
dtype: int64


In [101]:
newdata = newdata.dropna()
print(newdata.shape)
print(newdata.isnull().sum())

(2698696, 8)
PD_C        0
PD_NM       0
CLAC1_NM    0
CLAC2_NM    0
CLAC3_NM    0
CLNT_ID     0
SESS_ID     0
CLUSTER     0
dtype: int64


In [102]:
newdata

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_ID,SESS_ID,CLUSTER
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,1489284.0,2719478.0,1.0
1,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,4671890.0,2659813.0,3.0
2,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,559585.0,2330751.0,0.0
3,62282,여자 플라워덧신 2족선물세트 17403-2set,속옷/양말/홈웨어,여성양말류,여성일반양말,2820547.0,733100.0,0.0
8,58183,[17S/S] 울 자카드 져지 자켓 9317111001056 - 네이비 / 100,남성의류,남성의류아우터,남성캐주얼재킷,4339038.0,255409.0,1.0
...,...,...,...,...,...,...,...,...
3097359,705374,인디고 빨간머리앤 클리어파일 [클리어화일/서류보관/문서/일러스트화일/A4화일] - ...,문구/사무용품,일반문구/사무용품,파일/바인더,3140652.0,8968447.0,1.0
3097360,705374,인디고 빨간머리앤 클리어파일 [클리어화일/서류보관/문서/일러스트화일/A4화일] - ...,문구/사무용품,일반문구/사무용품,파일/바인더,5856443.0,271683.0,0.0
3097361,705374,인디고 빨간머리앤 클리어파일 [클리어화일/서류보관/문서/일러스트화일/A4화일] - ...,문구/사무용품,일반문구/사무용품,파일/바인더,5856443.0,271683.0,0.0
3097362,705374,인디고 빨간머리앤 클리어파일 [클리어화일/서류보관/문서/일러스트화일/A4화일] - ...,문구/사무용품,일반문구/사무용품,파일/바인더,3140652.0,8968447.0,0.0


In [114]:
new = newdata.groupby(['CLUSTER','PD_C']).agg({'PD_C':'size'})
new.tail

<bound method NDFrame.tail of                 PD_C
CLUSTER PD_C        
0.0     3          4
        4          1
        9          1
        11         4
        12         1
...              ...
4.0     846958     1
        847110     1
        847118     1
        847147     1
        847151     1

[792538 rows x 1 columns]>

In [113]:
new['rank'] = new[{'PD_C':'size'}].rank(method = 'max')
new[:20]

PD_C      rank
CLUSTER PD_C                
0.0     3        4  688304.0
        4        1  461839.0
        9        1  461839.0
        11       4  688304.0
        12       1  461839.0
        13       1  461839.0
        14       2  594831.0
        25       1  461839.0
        29       1  461839.0
        30       1  461839.0
        38       2  594831.0
        49       1  461839.0
        52      56  788823.0
        55       1  461839.0
        59       1  461839.0
        65       2  594831.0
        66       1  461839.0
        69       5  709491.0
        70      37  785702.0
        72       3  654807.0

In [ ]:
new2 = new.groupby()